In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os
from config import cfg
from scipy.stats import kde
from matplotlib.patches import Polygon

%matplotlib inline

In [2]:
path = "/mnt/storage/home/ja17618/scratch/DATA_DIR/context/urban/"
path2 = "/mnt/storage/home/ja17618/scratch/valid1/"

kitti_f = glob.glob(os.path.join(path, 'velodyne', '*.bin'))
sil_f = glob.glob(os.path.join(path2, 'output', '*.bin'))



In [3]:
def lidar_to_bird_view(x, y, factor=1):
    # using the cfg.INPUT_XXX
    a = (x - cfg.X_MIN) / cfg.VOXEL_X_SIZE * factor
    b = (y - cfg.Y_MIN) / cfg.VOXEL_Y_SIZE * factor
    a = np.clip(a, a_max=(cfg.X_MAX - cfg.X_MIN) / cfg.VOXEL_X_SIZE * factor, a_min=0)
    b = np.clip(b, a_max=(cfg.Y_MAX - cfg.Y_MIN) / cfg.VOXEL_Y_SIZE * factor, a_min=0)
    return a, b

In [4]:
def lidar_to_bird_view_img(lidar, factor=1):
    # Input:
    #   lidar: (N', 4)
    # Output:
    #   birdview: (w, l, 3)
    birdview = np.zeros(
        (cfg.INPUT_HEIGHT * factor, cfg.INPUT_WIDTH * factor, 1))
    for point in lidar:
        x, y = point[0:2]
        if cfg.X_MIN < x < cfg.X_MAX and cfg.Y_MIN < y < cfg.Y_MAX:
            x, y = int((x - cfg.X_MIN) / cfg.VOXEL_X_SIZE *
                       factor), int((y - cfg.Y_MIN) / cfg.VOXEL_Y_SIZE * factor)
            birdview[y, x] += 1
    birdview = birdview - np.min(birdview)
    divisor = np.max(birdview) - np.min(birdview)
    # TODO: adjust this factor
    birdview = np.clip((birdview / divisor * 255) *
                       5 * factor, a_min=0, a_max=255)
    birdview = np.tile(birdview, 3).astype(np.uint8)

    return birdview

In [5]:
def calculate_density(file):
    x_size=file.shape[0]
    y_size=file.shape[1]
    val_I = range(0, x_size, 1)
    val_J= range(0, y_size, 1)
    file[file > 1] = 1
    file[:][:][:,0]

    return file

In [10]:
kitti_d = []
for pcl in kitti_f[:1000]:
    pcl = np.fromfile(pcl,dtype=np.float32).reshape((-1,4))
    kitti_bev = lidar_to_bird_view_img(pcl,factor=cfg.BV_LOG_FACTOR)
    x_size,y_size = kitti_bev.shape[0],kitti_bev.shape[1]
    my_array = calculate_density(kitti_bev)
    kitti_d.append((np.sum(np.asarray(my_array))/3)/(x_size*y_size))

In [11]:
sil_d = []
for pcl1 in sil_f[:1000]:
    pcl1 = np.fromfile(pcl1,dtype=np.float32).reshape((-1,4))
    sil_bev = lidar_to_bird_view_img(pcl1,factor=cfg.BV_LOG_FACTOR)
    x_size,y_size = sil_bev.shape[0].sil_bev.shape[1]
    my_array = calculate_density(sil_bev)
    sil_d.append((np.sum(np.asarray(my_array))/3)/(x_size*y_size))

AttributeError: 'int' object has no attribute 'sil_bev'

In [ ]:
np.mean(kitti_d)

In [ ]:
np.mean(sil_d)

In [ ]:
(np.mean(kitti_d)-np.mean(sil_d))/np.mean(kitti_d) * 100


In [ ]:
kitti_d = np.array(kitti_d)
sil_d= np.array(sil_d)

In [ ]:
data = [kitti_d,sil_d]
dists = ['Kitti Point Cloud Density', 'SIL Point Cloud Density']
fig, ax1 = plt.subplots(figsize=(10, 10))
fig.canvas.set_window_title('A Boxplot Example')
plt.subplots_adjust(left=0.075, right=0.95, top=0.9, bottom=0.25)
numDists = 1
bp = plt.boxplot(data, notch=0, sym='+', vert=1, whis=1.5)
plt.setp(bp['boxes'], color='black')
plt.setp(bp['whiskers'], color='black')
plt.setp(bp['fliers'], color='red', marker='+')

# Add a horizontal grid to the plot, but make it very light in color
# so we can use it for reading data values but not be distracting
ax1.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',
               alpha=0.5)

# Hide these grid behind plot objects
ax1.set_axisbelow(True)
ax1.set_title('Density Comparison between SIL and KITTI Point Cloud Frames')
ax1.set_ylabel('Density')

# Now fill the boxes with desired colors
boxColors = ['darkkhaki', 'royalblue']
numBoxes = numDists*2
medians = list(range(numBoxes))
for i in range(numBoxes):
    box = bp['boxes'][i]
    boxX = []
    boxY = []
    for j in range(5):
        boxX.append(box.get_xdata()[j])
        boxY.append(box.get_ydata()[j])
    boxCoords = list(zip(boxX, boxY))
    # Alternate between Dark Khaki and Royal Blue
    k = i % 2
    boxPolygon = Polygon(boxCoords, facecolor=boxColors[k])
    ax1.add_patch(boxPolygon)
    # Now draw the median lines back over what we just filled in
    med = bp['medians'][i]
    medianX = []
    medianY = []
    for j in range(2):
        medianX.append(med.get_xdata()[j])
        medianY.append(med.get_ydata()[j])
        plt.plot(medianX, medianY, 'k')
        medians[i] = medianY[0]
    # Finally, overplot the sample averages, with horizontal alignment
    # in the center of each box
    plt.plot([np.mean(med.get_xdata())], [np.mean(data[i])],
             color='w', marker='X', markeredgecolor='k')
    x1 =[np.average(med.get_xdata())] 
    y1 =[np.average(data[i])]
    for a,b in zip(x1, y1): 
        st = "{0:.4f}".format(b)
        plt.text(a+0.09, b, st)
# Set the axes ranges and axes labels
ax1.set_xlim(0.5, numBoxes + 0.5)
xtickNames = plt.setp(ax1, xticklabels=dists)
plt.setp(xtickNames, rotation=45, fontsize=8)
#
# Due to the Y-axis scale being different across samples, it can be
# hard to compare differences in medians across the samples. Add upper
# X-axis tick labels with the sample medians to aid in comparison
# (just use two decimal places of precision)
pos = np.arange(numBoxes) + 1
upperLabels = [str(np.round(s, 4)) for s in medians]
weights = ['bold', 'semibold']
top=0.0025
for tick, label in zip(range(numBoxes), ax1.get_xticklabels()):
    k = tick % 2
    ax1.text(pos[tick], top, upperLabels[tick],
              horizontalalignment='center', 
              size='medium', weight=weights[k],
              color=boxColors[k])


# Finally, add a basic legend
# plt.figtext(0.80, 0.08, str(N) + ' Random Numbers',
#             backgroundcolor=boxColors[0], color='black', weight='roman',
#             size='x-small')

plt.figtext(0.80, 0.015, 'X', color='white', backgroundcolor='silver',
            weight='roman', size='medium')
plt.figtext(0.815, 0.013, ' Average Value', color='black', weight='roman',
            size='medium')
plt.savefig('density.png',dpi=300)
# plt.show()